In [1]:
import os

In [2]:
os.getcwd()

'd:\\Delivery-time-prediction\\experiment'

In [3]:
os.chdir("../")

In [4]:
os.getcwd()

'd:\\Delivery-time-prediction'

In [5]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, r2_score

In [7]:
df = pd.read_csv(r"experiment\finalTrain.csv")

In [8]:
df

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,0xcdcd,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55,22:10,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46
1,0xd987,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,14:55,15:05,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23
2,0x2784,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,04-03-2022,17:30,17:40,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21
3,0xc8b6,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,09:20,09:30,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20
4,0xdb64,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,19:50,20:05,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45579,0x7c09,JAPRES04DEL01,30.0,4.8,26.902328,75.794257,26.912328,75.804257,24-03-2022,11:35,11:45,Windy,High,1,Meal,motorcycle,0.0,No,Metropolitian,32
45580,0xd641,AGRRES16DEL01,21.0,4.6,0.000000,0.000000,0.070000,0.070000,16-02-2022,19:55,20:10,Windy,Jam,0,Buffet,motorcycle,1.0,No,Metropolitian,36
45581,0x4f8d,CHENRES08DEL03,30.0,4.9,13.022394,80.242439,13.052394,80.272439,11-03-2022,23:50,24:05:00,Cloudy,Low,1,Drinks,scooter,0.0,No,Metropolitian,16
45582,0x5eee,COIMBRES11DEL01,20.0,4.7,11.001753,76.986241,11.041753,77.026241,07-03-2022,13:35,13:40,Cloudy,High,0,Snack,motorcycle,1.0,No,Metropolitian,26


In [10]:
df = df.copy()

In [11]:
df['Order_Date'] = pd.to_datetime(df['Order_Date'] , format="%d-%m-%Y", errors="coerce")
df['Order_day'] = df['Order_Date'].dt.day
df['Order_month'] = df['Order_Date'].dt.month
df['Order_year'] = df['Order_Date'].dt.year
df = df.drop(columns=['Order_Date'],axis=1) 

In [24]:
def to_minute(x):
    t = pd.to_datetime(x,format="%H:%M", errors="coerce")
    if pd.isna(t):
        return 0
    return t.hour * 60 + t.minute
        

In [25]:
df['Time_Orderd']  =df['Time_Orderd'].apply(to_minute)

In [26]:
df['Time_Orderd']

0        0
1        0
2        0
3        0
4        0
        ..
45579    0
45580    0
45581    0
45582    0
45583    0
Name: Time_Orderd, Length: 45584, dtype: int64

In [28]:
df['Time_Order_picked'] = df['Time_Order_picked'].apply(to_minute)

In [29]:
def haversine(lat1, lon1, lat2, lon2):
        R = 6371
        lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
        return R * 2 * np.arcsin(np.sqrt(a))

In [30]:
df['delivery_distance_km'] = haversine(
            df['Restaurant_latitude'], df['Restaurant_longitude'],
            df['Delivery_location_latitude'], df['Delivery_location_longitude']
        )

In [ ]:
X = df.drop(columns=[
            'Restaurant_latitude','Restaurant_longitude',
            'Delivery_location_latitude','Delivery_location_longitude'
        ])
        
return X

In [ ]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.metrics import mean_absolute_error, r2_score , accuracy_score


df = pd.read_csv(r"experiment\finalTrain.csv") 

# ---------- Custom Feature Engineering ----------
class FeatureEngineer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        
        X = X.copy()
        
        # Convert Order_Date
        X['Order_Date'] = pd.to_datetime(X['Order_Date'], format="%d-%m-%Y", errors="coerce")
        X['Order_day'] = X['Order_Date'].dt.day
        X['Order_month'] = X['Order_Date'].dt.month
        X['Order_weekday'] = X['Order_Date'].dt.weekday
        X = X.drop(columns=['Order_Date'])
        
        # Convert Time columns → minutes since midnight
        def to_minutes(x):
            t = pd.to_datetime(x, format="%H:%M", errors="coerce")
            if pd.isna(t):
                return 0
            return t.hour * 60 + t.minute
        
        X['Time_Orderd'] = X['Time_Orderd'].apply(to_minutes)
        X['Time_Order_picked'] = X['Time_Order_picked'].apply(to_minutes)
        
        # Compute delivery distance
        def haversine(lat1, lon1, lat2, lon2):
            R = 6371
            lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
            dlat = lat2 - lat1
            dlon = lon2 - lon1
            a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
            return R * 2 * np.arcsin(np.sqrt(a))
        
        X['delivery_distance_km'] = haversine(
            X['Restaurant_latitude'], X['Restaurant_longitude'],
            X['Delivery_location_latitude'], X['Delivery_location_longitude']
        )
        
        # Drop raw lat/longs
        X = X.drop(columns=[
            'Restaurant_latitude','Restaurant_longitude',
            'Delivery_location_latitude','Delivery_location_longitude'
        ])
        
        return X

# ---------- Prepare Data ----------
X = df.drop(columns=['ID', 'Delivery_person_ID'])
y = X['Time_taken (min)']
X = X.drop(columns=['Time_taken (min)'])

# Identify feature types (before transformation)
numeric_features = ['Delivery_person_Age','Delivery_person_Ratings',
                    'Vehicle_condition','multiple_deliveries',
                    'Time_Orderd','Time_Order_picked',
                    'Order_day','Order_month','Order_weekday',
                    'delivery_distance_km']

categorical_features = ['Weather_conditions','Road_traffic_density',
                        'Type_of_order','Type_of_vehicle','Festival','City']

# ---------- Preprocessors ----------
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ],
    remainder="drop"
)

# ---------- Final Pipeline with Model ----------
model = Pipeline(steps=[
    ("features", FeatureEngineer()),
    ("preprocessor", preprocessor),
    ("regressor", RandomForestRegressor(
        n_estimators=200,
        max_depth=20,
        random_state=42,
        n_jobs=-1
    ))
])

# ---------- Train/Test Split ----------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Fit model
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluation
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("✅ Model trained successfully")
print("MAE:", mae)
print("R² Score:", r2)

# Optional: Cross-validation
cv_scores = cross_val_score(model, X, y, cv=5, scoring="r2")
print("CV R² Mean:", np.mean(cv_scores))


✅ Model trained successfully
MAE: 3.179068123379767
R² Score: 0.8166579793836771
CV R² Mean: 0.8187124838399118


In [36]:
accuracy_score(y_test,y_pred)

NameError: name 'accuracy_score' is not defined

In [37]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, r2_score
import lightgbm as lgb

# ---------- Load Dataset ----------
df = pd.read_csv(r"experiment\finalTrain.csv")  # replace with your dataset path

# ---------- Enhanced Feature Engineering ----------
class FeatureEngineer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        
        # Convert Order_Date
        X['Order_Date'] = pd.to_datetime(X['Order_Date'], format="%d-%m-%Y", errors="coerce")
        X['Order_day'] = X['Order_Date'].dt.day
        X['Order_month'] = X['Order_Date'].dt.month
        X['Order_weekday'] = X['Order_Date'].dt.weekday
        X['is_weekend'] = (X['Order_weekday'] >= 5).astype(int)
        X = X.drop(columns=['Order_Date'])
        
        # Convert Time columns → minutes since midnight
        def to_minutes(x):
            t = pd.to_datetime(x, format="%H:%M", errors="coerce")
            if pd.isna(t):
                return np.nan
            return t.hour * 60 + t.minute
        
        X['Time_Orderd'] = X['Time_Orderd'].apply(to_minutes)
        X['Time_Order_picked'] = X['Time_Order_picked'].apply(to_minutes)
        
        # Create time-to-pickup feature
        X['time_to_pickup'] = X['Time_Order_picked'] - X['Time_Orderd']
        X['time_to_pickup'] = X['time_to_pickup'].fillna(X['time_to_pickup'].median())
        
        # Extract order hour
        X['order_hour'] = X['Time_Orderd'] // 60
        
        # Compute delivery distance
        def haversine(lat1, lon1, lat2, lon2):
            R = 6371
            lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
            dlat = lat2 - lat1
            dlon = lon2 - lon1
            a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
            return R * 2 * np.arcsin(np.sqrt(a))
        
        X['delivery_distance_km'] = haversine(
            X['Restaurant_latitude'], X['Restaurant_longitude'],
            X['Delivery_location_latitude'], X['Delivery_location_longitude']
        )
        
        # Drop raw lat/longs
        X = X.drop(columns=[
            'Restaurant_latitude','Restaurant_longitude',
            'Delivery_location_latitude','Delivery_location_longitude'
        ])
        
        # Cyclical encoding for weekday
        X['weekday_sin'] = np.sin(2 * np.pi * X['Order_weekday']/7)
        X['weekday_cos'] = np.cos(2 * np.pi * X['Order_weekday']/7)
        
        return X

# ---------- Prepare Data ----------
X = df.drop(columns=['ID', 'Delivery_person_ID'])
y = X['Time_taken (min)']
X = X.drop(columns=['Time_taken (min)'])

# Identify feature types
numeric_features = [
    'Delivery_person_Age','Delivery_person_Ratings',
    'Vehicle_condition','multiple_deliveries',
    'Time_Orderd','Time_Order_picked','time_to_pickup','order_hour',
    'Order_day','Order_month','delivery_distance_km',
    'weekday_sin','weekday_cos'
]

categorical_features = [
    'Weather_conditions','Road_traffic_density',
    'Type_of_order','Type_of_vehicle','Festival','City','is_weekend'
]

# ---------- Preprocessors ----------
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ],
    remainder="drop"
)

# ---------- Final Pipeline with LightGBM ----------
model = Pipeline(steps=[
    ("features", FeatureEngineer()),
    ("preprocessor", preprocessor),
    ("regressor", lgb.LGBMRegressor(
        n_estimators=500,
        max_depth=20,
        learning_rate=0.05,
        random_state=42
    ))
])

# ---------- Train/Test Split ----------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Fit model
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluation
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("✅ Model trained successfully")
print("MAE:", mae)
print("R² Score:", r2)

# Optional: Cross-validation
cv_scores = cross_val_score(model, X, y, cv=5, scoring="r2")
print("CV R² Mean:", np.mean(cv_scores))


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009585 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 730
[LightGBM] [Info] Number of data points in the train set: 36467, number of used features: 38
[LightGBM] [Info] Start training from score 26.309677


c:\Users\codey\anaconda3\envs\delivery\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


✅ Model trained successfully
MAE: 3.181589188060781
R² Score: 0.8180275719373391
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003419 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 728
[LightGBM] [Info] Number of data points in the train set: 36467, number of used features: 38
[LightGBM] [Info] Start training from score 26.265775


c:\Users\codey\anaconda3\envs\delivery\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003193 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 729
[LightGBM] [Info] Number of data points in the train set: 36467, number of used features: 38
[LightGBM] [Info] Start training from score 26.297995


...

c:\Users\codey\anaconda3\envs\delivery\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003568 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 731
[LightGBM] [Info] Number of data points in the train set: 36467, number of used features: 38
[LightGBM] [Info] Start training from score 26.304769


c:\Users\codey\anaconda3\envs\delivery\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002994 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 729
[LightGBM] [Info] Number of data points in the train set: 36468, number of used features: 38
[LightGBM] [Info] Start training from score 26.314495
CV R² Mean: 0.82456818582364


c:\Users\codey\anaconda3\envs\delivery\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
